<p style="float: left;"><a href="variances.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="case-classes.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>

# Lower type bounds

We have seen [upper type bounds](upper-types-bounds.ipynb) for type parameters. 

- In a type parameter declaration such as `T <: U`, 
- the type parameter **`T` is restricted to range only over <span style="color:red">subtypes</span> of type `U`**.

Symmetrical to _upper bounds_ are _lower bounds_ in Scala. 

- In a type parameter declaration `T >: S`,
- the type parameter **`T` is restricted to range only over <span style="color:red">supertypes</span> of type `S`**.

Using _lower bounds_, we can generalize the `push` method in `Stack` as follows:

```scala
class Stack[+A] {
    def push[B >: A](elem: B): Stack[B] = . . .
}
```

Technically, this solves our variance problem since **now the type parameter `A` appears no longer
in contravariance position**, i.e., as a parameter type of method `push`. Instead, **`A` appears as _lower
type bound_ for another type parameter, `B`**.


We’ve fixed the variance issue and generalized `push`: **we can now add elements of type `B` (a supertype of `A`), and the returned stack’s type updates accordingly to `Stack[B]`.**

In [ ]:
abstract class Stack[+A]:
    def push[B >: A](x: B): Stack[B] = new NonEmptyStack[B](x, this)
    def isEmpty: Boolean
    def top: A
    def pop: Stack[A]

class EmptyStack[A] extends Stack[A]:
    def isEmpty = true
    def top = throw new Exception("EmptyStack.top")
    def pop = throw new Exception("EmptyStack.pop")
    override def toString: String = "--()"

class NonEmptyStack[A](elem: A, rest: Stack[A]) extends Stack[A]:
    def isEmpty = false
    def top = elem
    def pop = rest
    override def toString(): String =
        "--" ++ top.toString() ++ "--<" ++ pop.toString()

Not exactly… **Scala 3 introduces union types (`A | B`), so <span style="color:red"> when you push an element of type `B` onto a `Stack[A]`, the resulting stack has the type `Stack[A | B]` instead of `Stack[B]`</span>.**

In [15]:
val stackInt = new EmptyStack().push(1 : Int)

stackInt: Stack[Int] = --1--<--()

In [16]:
val stackDouble = stackInt.push(1 : Double)

stackDouble: Stack[ammonite.$sess.cmd15.wrapper.cmd8.Stack[scala.Int | scala.Double]] = --1.0--<--1--<--()

In [17]:
new EmptyStack().push(1: Int).push(1 : Double).push('c')

res17: Stack[cmd17.this.cmd8.Stack[scala.Int | scala.Double | scala.Char]] = --c--<--1.0--<--1--<--()

<p style="float: left;"><a href="variances.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="case-classes.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>